## 1. Install Required Libraries
Install IBM Watsonx, LangChain, and related dependencies for prompt engineering and LLM integration.

In [1]:
%%capture
!pip install "ibm-watsonx-ai==0.2.6"
!pip install "langchain==0.1.16" 
!pip install "langchain-ibm==0.1.4"

## 2. Import Libraries and Suppress Warnings
Import Watsonx, LangChain, and utility modules. Suppress warnings for cleaner output.

In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

## 3. Install OpenAI Python SDK
Install the OpenAI SDK to enable access to OpenAI LLMs.

In [15]:
!pip install openai

## 4. Define OpenAI LLM Wrapper
Create a function to call OpenAI's GPT-3.5-turbo model with custom parameters for prompt engineering.

In [ ]:
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key= "Your-OpenAI-API-Key")

def llm_model(prompt_txt, params=None):
    model_id = "gpt-3.5-turbo"  # or "gpt-3.5-turbo"

    default_params = {
        "max_tokens": 256,
        "temperature": 0.5,
        "top_p": 0.2,
    }

    if params:
        default_params.update({
            "max_tokens": params.get("max_new_tokens", default_params["max_tokens"]),
            "temperature": params.get("temperature", default_params["temperature"]),
            "top_p": params.get("top_p", default_params["top_p"]),
        })

    response = client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "user", "content": prompt_txt}
        ],
        max_tokens=default_params["max_tokens"],
        temperature=default_params["temperature"],
        top_p=default_params["top_p"]
    )

    return response.choices[0].message.content



## 5. Show Example Generation Parameters
Display example values for Watsonx generation parameters.

In [22]:
GenParams().get_example_values()

{'decoding_method': 'sample',
 'length_penalty': {'decay_factor': 2.5, 'start_index': 5},
 'temperature': 0.5,
 'top_p': 0.2,
 'top_k': 1,
 'random_seed': 33,
 'repetition_penalty': 2,
 'min_new_tokens': 50,
 'max_new_tokens': 200,
 'stop_sequences': ['fail'],
 ' time_limit': 600000,
 'truncate_input_tokens': 200,
 'return_options': {'input_text': True,
  'generated_tokens': True,
  'input_tokens': True,
  'token_logprobs': True,
  'token_ranks': False,
  'top_n_tokens': False}}

## 6. Generate Text with OpenAI LLM
Demonstrate text generation using the custom llm_model function and a simple prompt.

In [23]:
params = {
    "max_new_tokens": 128,
    "temperature": 0.5,
    "top_p": 0.2,
    "top_k": 1  # OpenAI still doesn’t support top_k, so it’s ignored
}

prompt = "The wind is"
response = llm_model(prompt, params)

print(f"prompt: {prompt}\n")
print(f"response: {response}\n")


prompt: The wind is

response: a natural phenomenon characterized by the movement of air from areas of high pressure to areas of low pressure. It is caused by differences in atmospheric pressure, which are primarily driven by temperature variations. Wind can vary in strength from a gentle breeze to a violent storm, and it plays a key role in weather patterns and climate conditions around the world. It also has significant impacts on various human activities, including aviation, sailing, and the generation of wind power.



## 7. Prompt-based Classification Example
Use a prompt to classify a statement as true or false using the LLM.

In [24]:
prompt = """Classify the following statement as true or false: 
            'The Eiffel Tower is located in Berlin.'

            Answer:
"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: Classify the following statement as true or false: 
            'The Eiffel Tower is located in Berlin.'

            Answer:


response : False



## 8. Prompt-based Translation Example
Translate an English sentence to French using a few-shot prompt and the LLM.

In [25]:
params = {
    "max_new_tokens": 20,
    "temperature": 0.1,
}

prompt = """Here is an example of translating a sentence from English to French:

            English: “How is the weather today?”
            French: “Comment est le temps aujourd'hui?”
            
            Now, translate the following sentence from English to French:
            
            English: “Where is the nearest supermarket?”
            
"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: Here is an example of translating a sentence from English to French:

            English: “How is the weather today?”
            French: “Comment est le temps aujourd'hui?”
            
            Now, translate the following sentence from English to French:
            
            English: “Where is the nearest supermarket?”
            


response : French: "Où est le supermarché le plus proche?"



## 9. Prompt-based Emotion Classification
Classify the emotion in a statement using a few-shot prompt and the LLM.

In [26]:
 #parameters  `max_new_tokens` to 10, which constrains the model to generate brief responses

params = {
    "max_new_tokens": 10,
}

prompt = """Here are few examples of classifying emotions in statements:

            Statement: 'I just won my first marathon!'
            Emotion: Joy
            
            Statement: 'I can't believe I lost my keys again.'
            Emotion: Frustration
            
            Statement: 'My best friend is moving to another country.'
            Emotion: Sadness
            
            Now, classify the emotion in the following statement:
            Statement: 'That movie was so scary I had to cover my eyes.’
            

"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: Here are few examples of classifying emotions in statements:

           Statement: 'I just won my first marathon!'
           Emotion: Joy
           
           Statement: 'I can't believe I lost my keys again.'
           Emotion: Frustration
           
           Statement: 'My best friend is moving to another country.'
           Emotion: Sadness
           
           Now, classify the emotion in the following statement:
           Statement: 'That movie was so scary I had to cover my eyes.’
           



response : Emotion: Fear



## 10. Prompt-based Step-by-step Reasoning
Ask the LLM to break down a math problem into step-by-step reasoning.

In [27]:
params = {
    "max_new_tokens": 512,
    "temperature": 0.5,
}

prompt = """Consider the problem: 'A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples. 
            How many apples are there now?’

            Break down each step of your calculation

"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: Consider the problem: 'A store had 22 apples. They sold 15 apples today and got a new delivery of 8 apples. 
            How many apples are there now?’

            Break down each step of your calculation



response : Step 1: Start with the initial number of apples, which is 22.

Step 2: Subtract the number of apples sold, which is 15. So, 22 - 15 = 7.

Step 3: Add the number of apples delivered, which is 8. So, 7 + 8 = 15.

So, there are now 15 apples in the store.



## 11. Prompt-based Multi-step Reasoning
Request the LLM to solve a riddle with multiple independent calculations and explanations.

In [28]:
params = {
    "max_new_tokens": 512,
}

prompt = """When I was 6, my sister was half of my age. Now I am 70, what age is my sister?

            Provide three independent calculations and explanations, then determine the most consistent result.

"""
response = llm_model(prompt, params)
print(f"prompt: {prompt}\n")
print(f"response : {response}\n")

prompt: When I was 6, my sister was half of my age. Now I am 70, what age is my sister?

            Provide three independent calculations and explanations, then determine the most consistent result.



response : Calculation 1:
When I was 6, my sister was half my age, which means she was 3 years old. So, my sister is 3 years younger than me. If I am now 70, my sister would be 70 - 3 = 67 years old.

Calculation 2:
The age difference between my sister and me has always been the same because we both grow older at the same rate. When I was 6, she was 3, so the age difference is 3 years. Therefore, if I am 70 now, she must be 70 - 3 = 67 years old.

Calculation 3:
If we consider the time when I was 6 and my sister was 3, it has been 64 years since then (70 - 6 = 64). So, my sister, who was 3 at that time, would have aged 64 years as well, making her 3 + 64 = 67 years old.

The most consistent result from all three calculations is that my sister is 67 years old.



## 12. Define a Simple OpenAI LLM Function
Create a simple function to invoke OpenAI's GPT-3.5-turbo for text generation.

In [29]:
from openai import OpenAI


# Define the OpenAI model and generation parameters
model_id = "gpt-3.5-turbo"  # or "gpt-3.5-turbo" depending on your needs

parameters = {
    "max_tokens": 256,       # controls max tokens in output
    "temperature": 0.5       # controls creativity/randomness
}

# This is the equivalent of `mixtral_llm.invoke(prompt)` in Watsonx
def mixtral_llm(prompt):
    response = client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=parameters["max_tokens"],
        temperature=parameters["temperature"]
    )
    return response.choices[0].message.content

# Example usage
response = mixtral_llm("The wind is")
print(response)


blowing gently, rustling the leaves on the trees and making the tall grass sway. It carries the scent of the ocean, fresh and salty, mixed with the earthy smell of the forest. The wind is a messenger, bringing whispers of distant places, carrying stories from the other side of the world. It's a soothing sound, a natural lullaby that makes your eyes grow heavy. The wind is a painter, it shapes the clouds in the sky, it stirs the sea into white tipped waves. It's a powerful force, yet it can be as gentle as a soft caress against your skin. The wind is ever changing, sometimes a gentle breeze, sometimes a raging storm. It's a constant reminder of nature's beauty and power.


## 13. Create a Prompt Template with LangChain
Define a prompt template for generating jokes using LangChain's PromptTemplate.

In [30]:
template = """Tell me a {adjective} joke about {content}.
"""
prompt = PromptTemplate.from_template(template)
prompt 

PromptTemplate(input_variables=['adjective', 'content'], template='Tell me a {adjective} joke about {content}.\n')

## 14. Format Prompt Template
Format the prompt template with specific variables to generate a custom prompt.

In [31]:
prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.\n'

## 15. Upgrade LangChain and OpenAI
Upgrade LangChain and OpenAI packages to the latest versions.

In [36]:
!pip install --upgrade langchain openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.3/433.3 kB 27.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.53
    Uninstalling langchain-core-0.1.53:
      Successfully uninstalled langchain-core-0.1.53
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.0.2
    Uninstalling langchain-text-splitters-0.0.2:
      Successfully uninstalled langchain-text-splitters-0.0.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.16
    Uninstalling langchain-0.1.16:
      Successfully uninstalled langchain-0.1.16
ERROR: pip's dependency reso

## 16. LangChain LLMChain Example
Use LangChain's LLMChain to generate a story with a prompt template and OpenAI LLM.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# 1. Define your prompt template
prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="Write a {adjective} story about {content}."
)

# 2. Set up OpenAI model via LangChain
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # or "gpt-3.5-turbo"
    temperature=0.5,
    max_tokens=256,
    openai_api_key="Your-OpenAI-API-Key"  # Replace with your OpenAI API key
)

# 3. Wrap it in an LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 4. Invoke the chain with inputs
response = llm_chain.invoke({"adjective": "funny", "content": "chickens"})

# 5. Print the result
print(response["text"])


Once upon a time, in a small farmyard, there lived a group of chickens who were known for their mischievous antics. One day, the chickens decided to play a prank on the farmer by hiding all of his tools in their coop. The farmer searched high and low, but couldn't find his tools anywhere.

Frustrated, the farmer turned to the chickens and asked if they knew where his tools were. The chickens just clucked innocently and went about their business. But as soon as the farmer turned his back, the chickens burst into laughter and started clucking loudly, giving away their secret.

The farmer finally caught on to their prank and decided to teach the chickens a lesson. He set up a fake scarecrow in the middle of the field and dressed it up in overalls and a straw hat, just like himself. The chickens, thinking it was the farmer, were terrified and refused to come out of their coop.

But the farmer wasn't done yet. He enlisted the help of a sneaky fox who was known for his cunning ways. The fox 

## 17. Invoke LLMChain with New Inputs
Invoke the LLMChain with different variables to generate a new story.

In [43]:
response = llm_chain.invoke(input = {"adjective": "sad", "content": "fish"})
print(response["text"])

In a small, murky pond lived a beautiful school of fish. They swam gracefully through the water, their scales shimmering in the sunlight. But as time passed, the pond began to dry up due to a lack of rain. The water level dropped lower and lower, leaving the fish struggling to survive.

One by one, the fish started to die off. Their once vibrant colors faded, their bodies becoming weak and frail. The leader of the school, a wise old fish named Finn, tried his best to keep everyone together and hopeful. But as the days went by, more and more fish succumbed to the harsh conditions.

Finn watched helplessly as his friends and family members died around him. He felt a deep sense of sorrow and despair, knowing that there was nothing he could do to save them. The once lively pond was now filled with the lifeless bodies of his loved ones.

As Finn swam alone in the dwindling pond, he couldn't help but feel a sense of emptiness. He missed the laughter and chatter of his fellow fish, the sense 

## 18. Summarization with Prompt Template
Use a prompt template to summarize a passage in one sentence using LangChain and OpenAI.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 1. Your input content
content = """
    The rapid advancement of technology in the 21st century has transformed various industries, including healthcare, education, and transportation. 
    Innovations such as artificial intelligence, machine learning, and the Internet of Things have revolutionized how we approach everyday tasks and complex problems. 
    For instance, AI-powered diagnostic tools are improving the accuracy and speed of medical diagnoses, while smart transportation systems are making cities more efficient and reducing traffic congestion. 
    Moreover, online learning platforms are making education more accessible to people around the world, breaking down geographical and financial barriers. 
    These technological developments are not only enhancing productivity but also contributing to a more interconnected and informed society.
"""

# 2. Create the prompt template
template = """Summarize the following content in one sentence:\n\n{content}"""
prompt = PromptTemplate.from_template(template)

# 3. Initialize the OpenAI LLM (Runnable)
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # or "gpt-3.5-turbo"
    temperature=0.5,
    max_tokens=100,
    openai_api_key="your-openai-api-key"  # Replace with your OpenAI API key
)

# 4. Build the chain
llm_chain = LLMChain(prompt=prompt, llm=llm)

# 5. Run the chain
response = llm_chain.invoke({"content": content})

# 6. Output the result
print(response["text"])


The rapid technological advancements in the 21st century have revolutionized industries like healthcare, education, and transportation through innovations like artificial intelligence, machine learning, and the Internet of Things, improving efficiency, accessibility, and connectivity in society.


## 19. Question Answering with Prompt Template
Answer a question based on provided content using a prompt template and LangChain.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 1. The source content
content = """
    The solar system consists of the Sun, eight planets, their moons, dwarf planets, and smaller objects like asteroids and comets. 
    The inner planets—Mercury, Venus, Earth, and Mars—are rocky and solid. 
    The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and gaseous.
"""

# 2. The question to answer
question = "Which planets in the solar system are rocky and solid?"

# 3. Define the prompt template
template = """
Answer the question: "{question}" based on the following content:

{content}

Respond "Unsure about answer" if not sure about the answer.

Answer:
"""

prompt = PromptTemplate.from_template(template)

# 4. Set up the OpenAI model
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # or "gpt-3.5-turbo"
    temperature=0.2,
    max_tokens=100,
    openai_api_key="Your-OpenAI-API-Key"  # Replace with your OpenAI API key
)

# 5. Create the chain with custom output key
llm_chain = LLMChain(prompt=prompt, llm=llm, output_key="answer")

# 6. Run the chain with inputs
response = llm_chain.invoke({"question": question, "content": content})

# 7. Print the extracted answer
print(response["answer"])


Mercury, Venus, Earth, and Mars are the rocky and solid planets in the solar system.


## 20. Text Classification with Prompt Template
Classify a text into one of several categories using a prompt template and LangChain.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 1. Input text to classify
text = """
    The concert last night was an exhilarating experience with outstanding performances by all artists.
"""

# 2. The list of available categories
categories = "Entertainment, Food and Dining, Technology, Literature, Music."

# 3. Define the classification prompt template
template = """
Classify the following text into one of the given categories:

Text:
{text}

Categories:
{categories}

Respond with the single best-matching category.

Category:
"""

prompt = PromptTemplate.from_template(template)

# 4. Initialize the OpenAI model via LangChain
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # or "gpt-3.5-turbo"
    temperature=0.3,
    max_tokens=50,
    openai_api_key="Your-OpenAI-API-Key"  # Replace with your OpenAI API key
)

# 5. Build the classification chain with a custom output key
llm_chain = LLMChain(prompt=prompt, llm=llm, output_key="category")

# 6. Run the chain
response = llm_chain.invoke({"text": text, "categories": categories})

# 7. Output the classified category
print(response["category"])


Music


## 21. SQL Query Generation with Prompt Template
Generate an SQL query from a natural language description using LangChain and OpenAI.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 1. Natural language description of the SQL query
description = """
Retrieve the names and email addresses of all customers from the 'customers' table who have made a purchase in the last 30 days. 
The table 'purchases' contains a column 'purchase_date'
"""

# 2. Prompt template for SQL generation
template = """
Generate an SQL query based on the following description:

{description}

SQL Query:
"""

prompt = PromptTemplate.from_template(template)

# 3. OpenAI LLM setup with gpt-3.5-turbo
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.2,
    max_tokens=150,
    openai_api_key="Your-OpenAI-API-Key"  # Replace with your OpenAI API key
)

# 4. LLMChain with custom output key
llm_chain = LLMChain(prompt=prompt, llm=llm, output_key="query")

# 5. Run the chain
response = llm_chain.invoke({"description": description})

# 6. Print the generated SQL
print(response["query"])


SELECT c.name, c.email
FROM customers c
JOIN purchases p ON c.customer_id = p.customer_id
WHERE p.purchase_date >= DATE_SUB(NOW(), INTERVAL 30 DAY);


## 22. Dynamic Persona Q&A with Prompt Template
Answer a question in the style of a specific persona and tone using LangChain and OpenAI.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 1. Define the persona role and tone
role = """
    game master
"""

tone = "engaging and immersive"

# 2. Prompt template for dynamic persona Q&A
template = """
You are an expert {role}. I have this question: "{question}". I would like our conversation to be {tone}.

Answer:
"""

prompt = PromptTemplate.from_template(template)

# 3. Initialize OpenAI model via LangChain
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.8,
    max_tokens=200,
    openai_api_key="Your-OpenAI-API-Key"  # Replace with your OpenAI API key
)

# 4. Create the LLMChain with output_key "answer"
llm_chain = LLMChain(prompt=prompt, llm=llm, output_key="answer")

# 5. Example usage
question = "How should I describe a mysterious forest to new players in a DnD campaign?"
response = llm_chain.invoke({"role": role, "tone": tone, "question": question})

# 6. Print the answer
print(response["answer"])


Ah, describing a mysterious forest to new players in a DnD campaign can really set the tone for their adventure. Close your eyes and imagine a dense canopy of ancient trees, their branches intertwining overhead to create a shadowy, almost oppressive atmosphere. The air is thick with the earthy scent of moss and decaying leaves, and the only sound is the gentle rustling of leaves in the wind.

As your players step cautiously along the winding path, they catch glimpses of strange, glowing mushrooms and twisted roots that seem to reach out towards them. The sunlight filters through the dense foliage in eerie patterns, casting long shadows that seem to move of their own accord.

Suddenly, a faint whispering sound echoes through the trees, sending a shiver down their spines. Is it just the wind, or is there something more sinister lurking in the darkness? The feeling of being watched is palpable, and the players can't help but feel a sense of unease as they venture deeper into


## 23. Interactive Q&A Loop
Create an interactive loop to ask questions and get persona-based answers from the LLM.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 1. Define the persona role and tone
role = """
    game master
"""

tone = "engaging and immersive"

# 2. Initialize OpenAI model via LangChain
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.8,
    max_tokens=200,
    openai_api_key="Your-OpenAI-API-Key"  # Replace with your OpenAI API key
)

# 3. Create the LLMChain with output_key "answer"
llm_chain = LLMChain(prompt=prompt, llm=llm, output_key="answer")

# 4. Interactive Q&A loop
while True:
    query = input("Question: ")
    
    if query.lower() in ["quit","exit","bye"]:
        print("Answer: Goodbye!")
        break
        
    response = llm_chain.invoke(input = {"role": role, "question": query, "tone": tone})
    
    print("Answer: ", response["answer"])

Question:  Who are you?


Answer:  Ah, adventurer, that is a question that holds much weight in the realm of mysteries and legends. I am the keeper of tales, the weaver of destinies, the one who guides you through the intricate webs of fate. But who am I truly? That is a question only you can uncover through your journey with me. So, tell me, what is it that drives you, that fuels your quest for answers and adventure? Let us embark on this journey together and discover the truth behind the shadows.


Question:  bye


Answer: Goodbye!


## 24. Custom Generation Parameters Example
Demonstrate how changing generation parameters affects the LLM's output.

In [54]:
params = {
    "max_new_tokens": 128,
    "min_new_tokens": 100,
    "temperature": 1,
    "top_p": 0.1,
    "top_k": 1
}

prompt = "The wind is"

response = llm_model(prompt, params)
print(response)

 blowing fiercely, whipping through the trees and sending leaves swirling through the air. The sound of it howling and whistling is almost deafening, making it difficult to hear anything else. The force of the wind is strong enough to make it hard to walk against, pushing and pulling at my clothes and hair. Despite the chill in the air, the wind's power is invigorating, reminding me of the raw strength of nature.


## 25. Question Answering with Custom Prompt
Answer a question based on content using a custom prompt template and verbose output.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 1. Source content
content = """
The solar system consists of the Sun, eight planets, their moons, dwarf planets, and smaller objects like asteroids and comets. 
The inner planets—Mercury, Venus, Earth, and Mars—are rocky and solid. 
The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and gaseous.
"""

# 2. The question to answer
question = "Which planets in the solar system are rocky and solid?"

# 3. Prompt template
template = """
Answer the following question based on the given content.

Content:
{content}

Question:
{question}

Respond "Unsure about answer" if not sure about the answer.

Answer:
"""

prompt = PromptTemplate.from_template(template)

# 4. Setup the OpenAI model (gpt-3.5-turbo)
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.3,
    max_tokens=150,
    openai_api_key="your-openai-api-key"  # Replace with your OpenAI API key
)

# 5. Create the chain with verbose output
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    output_key="answer",
    verbose=True
)

# 6. Invoke with inputs
response = llm_chain.invoke({
    "question": question,
    "content": content
})

# 7. Output the answer
print(response["answer"])




> Entering new LLMChain chain...
Prompt after formatting:

Answer the following question based on the given content.

Content:

The solar system consists of the Sun, eight planets, their moons, dwarf planets, and smaller objects like asteroids and comets. 
The inner planets—Mercury, Venus, Earth, and Mars—are rocky and solid. 
The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and gaseous.


Question:
Which planets in the solar system are rocky and solid?

Respond "Unsure about answer" if not sure about the answer.

Answer:


> Finished chain.
The inner planets - Mercury, Venus, Earth, and Mars - are rocky and solid.


## 26. Few-shot Classification Example
Demonstrate few-shot learning by providing an example and then classifying a new text.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 1. Example for few-shot guidance
example_text = """
Last week's book fair was a delightful gathering of authors and readers, featuring discussions and book signings.
"""
example_category = "Literature"

# 2. Text to classify
text = """
The concert last night was an exhilarating experience with outstanding performances by all artists.
"""

# 3. Category options
categories = "Entertainment, Food and Dining, Technology, Literature, Music."

# 4. Prompt template for few-shot classification
template = """
Example:
Text: {example_text}
Category: {example_category}

Now, classify the following text into one of the specified categories:
{categories}

Text: {text}

Category:
"""

prompt = PromptTemplate.from_template(template)

# 5. Initialize OpenAI model via LangChain
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.3,
    max_tokens=50,
    openai_api_key="your-openai-api-key"  # Replace with your OpenAI API key
)

# 6. Build the chain
llm_chain = LLMChain(prompt=prompt, llm=llm, output_key="category")

# 7. Run the chain with input variables
response = llm_chain.invoke({
    "example_text": example_text,
    "example_category": example_category,
    "categories": categories,
    "text": text
})

# 8. Output the result
print(response["category"])


Music


## 27. (Optional) Add Your Own Experiments
Use this cell for your own prompt engineering or LLM experiments.